In [144]:
# Convert raw counts to TPM
# gene length from file  /Users/galebabu-ali/Documents/CPTAC-PDA/Data/Annotations/gencode.gene.info.v22.tsv column 'exon_length' 
# match gene names from raw counts file /Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_RAW_counts_matched_linkedomics_tumor.csv column 'gene_name'

def raw_counts_to_tpm(raw_counts, gene_lengths):
    # Convert gene lengths from bases to kilobases
    gene_lengths_kb = gene_lengths / 1000.0
    
    # Calculate RPK (Reads Per Kilobase)
    rpk = raw_counts.div(gene_lengths_kb, axis=0)
    
    # Calculate scaling factor
    scaling_factor = rpk.sum(axis=0) / 1e6
    
    # Calculate TPM
    tpm = rpk.div(scaling_factor, axis=1)
    
    return tpm

# Load counts and gene lengths
raw_counts = pd.read_csv('/Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_RAW_counts_matched_linkedomics_tumor.csv', index_col=0)
gene_info = pd.read_csv('/Users/galebabu-ali/Documents/CPTAC-PDA/Data/Annotations/gencode.gene.info.v22.tsv', sep='\t')

gene_lengths = gene_info.loc[:, ['gene_name', 'exon_length']]
gene_lengths = gene_lengths.sort_values(by='exon_length', ascending=False).drop_duplicates('gene_name')
# how many gene_names duplicated in gene_lengths
gene_lengths = gene_lengths.set_index('gene_name')['exon_length']

# overlap in gene names between raw_counts and gene_lengths
overlap_genes = raw_counts.index.intersection(gene_lengths.index)

# subset raw_counts and gene_lengths to overlapping genes only
raw_counts = raw_counts.loc[raw_counts.index.isin(overlap_genes)]
gene_lengths = gene_lengths.loc[overlap_genes]
# Align gene_lengths with raw_counts gene names
gene_lengths = gene_lengths.reindex(raw_counts.index)

# # Run TPM normalization
tpm_data = raw_counts_to_tpm(raw_counts, gene_lengths)
print("TPM normalization complete. Shape:", tpm_data.shape)

# # Save TPM data to CSV
tpm_data.to_csv('/Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_normalized_linkedomics_tumor.csv')
print("TPM data saved to /Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_normalized_linkedomics_tumor.csv")

# # log2 transform TPM data
tpm_data_log2 = np.log2(tpm_data + 1)
tpm_data_log2.to_csv('/Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_log2_normalized_linkedomics_tumor.csv')
print("Log2 transformed TPM data saved to /Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_log2_normalized_linkedomics_tumor.csv")



TPM normalization complete. Shape: (23213, 140)
TPM data saved to /Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_normalized_linkedomics_tumor.csv
Log2 transformed TPM data saved to /Users/galebabu-ali/Documents/CPTAC-PDA/Data/PDAC_tx_TPM_log2_normalized_linkedomics_tumor.csv


In [142]:
tpm_data.head()

,C3N-03884,C3L-04027,C3N-04283,C3L-04475,C3N-00517,C3L-01037,C3N-00302,C3L-01637,C3N-01382,C3N-01714,C3N-01381,C3L-01052,C3L-02112,C3L-00599,C3L-00640,C3N-01011,C3L-00881,C3N-03069,C3L-01689,C3N-03440,C3L-01051,C3N-02573,C3L-04072,C3L-01703,C3L-03630,C3L-00622,C3N-00518,C3N-02944,C3N-03061,C3N-02971,C3L-00589,C3N-03190,C3N-02585,C3L-02115,C3N-00511,C3L-04473,C3L-04853,C3L-02809,C3N-01165,C3N-01380,C3L-00819,C3L-01971,C3N-03780,C3L-04479,C3L-02606,C3L-04080,C3L-01054,C3L-02610,C3N-00513,C3L-03635,C3N-03211,C3L-00277,C3L-03123,C3N-00436,C3N-01502,C3N-01997,C3L-01453,C3N-01388,C3L-00625,C3L-01124,C3L-03639,C3N-00512,C3L-02899,C3L-03743,C3N-00516,C3L-03395,C3N-02940,C3N-03000,C3L-00017,C3L-01662,C3L-02890,C3N-02295,C3N-02592,C3L-01598,C3L-01328,C3N-01168,C3N-01716,C3N-04119,C3N-03006,C3L-02116,C3N-01719,C3N-02998,C3N-01167,C3N-01166,C3N-02768,C3N-01998,C3L-01687,C3N-02589,C3N-02010,C3N-03839,C3L-03632,C3L-00189,C3N-00514,C3L-01036,C3L-00928,C3L-03356,C3L-01053,C3L-00598,C3N-03007,C3L-04848,C3N-00303,C3L-02613,C3N-04282,C3N-01375,C3N-01715,C3N-03426,C3L-03628,C3N-02579,C3N-04126,C3N-03439,C3N-03430,C3N-03428,C3N-03173,C3L-02118,C3N-00198,C3L-02109,C3N-00957,C3L-04495,C3N-00249,C3L-00401,C3L-02463,C3L-00102,C3N-03086,C3N-03665,C3L-03394,C3N-03754,C3N-01012,C3N-03670,C3N-03039,C3L-03388,C3L-02897,C3N-00709,C3N-04284,C3N-03666,C3L-02604,C3N-01383,C3L-02701,C3N-03853,C3L-01031,C3N-03840
gene_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
RN7SL1,231339.522809,264582.339037,355673.278772,244299.218632,411755.142392,274197.175969,294425.137014,306863.090945,395295.273391,484331.530476,280320.209772,423781.435096,343436.697488,336983.008748,365075.932746,444792.679695,326782.259154,206447.668661,237300.213853,320929.802625,355277.043102,284749.067922,241773.227331,423272.809447,278826.566455,299889.828033,374563.797469,302517.914705,244485.932398,290923.681007,342294.660870,267852.277166,207414.560133,364577.951234,315304.798519,364570.202843,304343.495534,206430.057127,365336.856751,362375.873105,388946.711538,282075.886151,210806.932406,314705.423055,333366.244371,184341.955136,447009.269046,499898.152710,355296.226620,320723.093935,202121.527488,309613.029979,293317.876793,361230.154187,455757.572019,407074.045548,283987.323711,329770.006626,321809.167252,277812.292143,256332.048094,468309.523282,245997.000842,261567.439729,326365.482098,330968.439659,322375.115571,248913.837720,362870.505603,308404.206895,342572.423862,235598.389195,182481.175205,399073.460958,505851.345256,353261.084489,428657.165226,317416.874920,290041.290968,402743.098061,298505.672126,304110.593383,306580.661649,411428.487506,340281.959820,475351.558260,273194.063609,288388.223234,272510.767093,315464.412313,333434.191647,359743.150829,360112.004911,397639.165261,367076.685700,203131.168462,442245.331265,310817.614609,281353.405484,182183.415029,306544.493042,389841.864136,230389.815596,345466.967011,450585.084351,159312.690319,294084.872550,335411.967372,402009.389048,330111.132408,242239.044979,323204.065881,199206.010243,357092.226860,352172.503753,299731.514875,336595.513820,145826.918641,441593.850171,434283.306235,308393.598171,276459.693753,238423.262953,183130.526927,172184.395506,193263.173377,278080.875352,195105.591290,208319.157288,246973.416660,191803.158425,327383.216056,303284.941245,197785.884187,189076.594914,239647.596928,335231.423596,348717.700923,361374.804067,309877.183254
PIGR,89.441770,4.786846,196.855654,20027.921275,195.181247,337.413502,201.195292,258.526286,344.913305,445.554212,342.407798,114.198645,9393.798037,345.112801,1830.586116,683.710478,542.696559,63.490563,14472.375631,202.570242,979.749697,1582.523708,298.062570,1.094340,3232.530740,293.203313,789.189475,282.328953,530.868049,51.750100,1981.562122,135.931164,265.679934,367.191896,1347.898259,413.959161,451.821919,26.265072,713.625094,11.007590,232.906636,958.956530,283.890724,536

In [143]:
# print raw_counts data for genes RN7SL1 PIGR MALAT1 EEF1A1 LYZ
raw_counts.loc[['RN7SL1', 'PIGR', 'MALAT1', 'EEF1A1', 'LYZ']]

,C3N-03884,C3L-04027,C3N-04283,C3L-04475,C3N-00517,C3L-01037,C3N-00302,C3L-01637,C3N-01382,C3N-01714,C3N-01381,C3L-01052,C3L-02112,C3L-00599,C3L-00640,C3N-01011,C3L-00881,C3N-03069,C3L-01689,C3N-03440,C3L-01051,C3N-02573,C3L-04072,C3L-01703,C3L-03630,C3L-00622,C3N-00518,C3N-02944,C3N-03061,C3N-02971,C3L-00589,C3N-03190,C3N-02585,C3L-02115,C3N-00511,C3L-04473,C3L-04853,C3L-02809,C3N-01165,C3N-01380,C3L-00819,C3L-01971,C3N-03780,C3L-04479,C3L-02606,C3L-04080,C3L-01054,C3L-02610,C3N-00513,C3L-03635,C3N-03211,C3L-00277,C3L-03123,C3N-00436,C3N-01502,C3N-01997,C3L-01453,C3N-01388,C3L-00625,C3L-01124,C3L-03639,C3N-00512,C3L-02899,C3L-03743,C3N-00516,C3L-03395,C3N-02940,C3N-03000,C3L-00017,C3L-01662,C3L-02890,C3N-02295,C3N-02592,C3L-01598,C3L-01328,C3N-01168,C3N-01716,C3N-04119,C3N-03006,C3L-02116,C3N-01719,C3N-02998,C3N-01167,C3N-01166,C3N-02768,C3N-01998,C3L-01687,C3N-02589,C3N-02010,C3N-03839,C3L-03632,C3L-00189,C3N-00514,C3L-01036,C3L-00928,C3L-03356,C3L-01053,C3L-00598,C3N-03007,C3L-04848,C3N-00303,C3L-02613,C3N-04282,C3N-01375,C3N-01715,C3N-03426,C3L-03628,C3N-02579,C3N-04126,C3N-03439,C3N-03430,C3N-03428,C3N-03173,C3L-02118,C3N-00198,C3L-02109,C3N-00957,C3L-04495,C3N-00249,C3L-00401,C3L-02463,C3L-00102,C3N-03086,C3N-03665,C3L-03394,C3N-03754,C3N-01012,C3N-03670,C3N-03039,C3L-03388,C3L-02897,C3N-00709,C3N-04284,C3N-03666,C3L-02604,C3N-01383,C3L-02701,C3N-03853,C3L-01031,C3N-03840
gene_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
RN7SL1,324847,776800,1317548,755937,3013417,957067,821546,1244122,1538318,3094016,1020068,2179484,1329344,1057196,995579,2202320,1071973,919313,754924,1387520,1404797,812427,972823,1511426,894670,1133196,1377814,961346,835981,802403,1296567,1202563,580382,2317570,925884,1171907,1053426,487626,1469748,1435384,1698628,860244,1098777,968297,1056688,511021,2726440,2960920,1360719,1088967,468907,1082891,670754,1551433,1601581,1825865,805128,1118861,877358,765961,811186,1941146,704999,1036002,1077578,1328637,1234479,730140,1287024,1319970,1229969,562836,488285,1601628,2305883,1444555,1933344,1054360,1252792,1677182,1169242,982553,1385161,1672744,1297236,2271526,834203,1061790,836183,1099417,1119358,1099927,1424895,1129537,1111802,546323,2704553,948428,879304,409638,897574,1492366,750972,1374583,2190566,205793,853280,1790835,1656839,1106008,328979,1000857,456866,1378022,1243903,749259,1100949,305725,1737324,1587458,1004473,855050,676053,239530,310165,542661,1679525,552585,455182,699961,648171,865198,1475397,474538,601340,596482,1270236,889094,1649535,940539
PIGR,1832,205,10637,903973,20836,17179,8189,15289,19579,41518,18175,8567,530381,15793,72818,49380,25968,4124,671585,12775,56509,65861,17494,57,151296,16161,42345,13087,26478,2082,109486,8902,10844,34048,57735,19410,22812,905,41877,636,14837,42659,21584,240904,7242,715268,13577,27778,22449,3490,20032,15380,1345,117600,23408,22254,1237,14362,13939,76973,2288,26001,6037,1112445,15541,21417,66853,140,370297,14503,65387,12918,634,73186,2090,46861,10405,13035,25273,124322,22687,23982,45072,66229,7920,42804,23603,35553,14625,130746,36048,11538,35991,15556,28717,81763,13728,1641,11247,11171,891,3993,2219,14594,108,51967,39405,30225,15309,40235,974,12296,404403,35580,170001,15349,40365,36545,284294,10275,31702,52101,98075,12923,11567,30988,40377,28559,14687,63189,15097,29666,7486,87869,34465,18589,9760,26438,56145,44391
MALAT1,316775,882017,435267,416596,520860,782711,724651,488632,345479,589784,474568,435377,524195,517624,410413,396172,525720,311648,600611,500307,347161,602536,638084,504868,403743,420675,198269,522624,517656,381557,220526,541691,445402,405908,388947,426445,467671,385548,543519,392572,584380,456101,245526,457493,506185,509003,349766,406461,245221,515338,340726,557578,410854,286472,599494,415023,453070,313796,508914,619967,511489,477800,633915,373003,424372,468990,485503,492700,336097,327129,751780,501950,486322,509081,649429,437831,578282,411005,469459